In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install pathlib
%pip install scikit-learn
%pip install imbalanced-learn
%pip install scipy

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [26]:
ruta = Path("./Dataset/")

df = pd.concat(
    (pd.read_csv(file) for file in ruta.glob("*.csv")),
    ignore_index=True
)

In [27]:
df.columns = (df.columns.str.strip().str.lower().str.replace(" ", "_"))

In [28]:
import re
df["label"] = df["label"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s-]', '', x)).str.strip()

In [ ]:
#1. Manejo de valores negativos (inconsistentes para features de Dataset)

#Se inicia contando la cantidad de negativos por feature en el dataset

num_cols = df.select_dtypes(include="number").columns

neg_conteo = (df[num_cols] < 0).sum().sort_values(ascending=False)

neg_conteo.name="Conteo de Negativos por Feature"

neg_conteo[neg_conteo > 0]

,Conteo de Negativos por Feature
init_win_bytes_backward,1441552
init_win_bytes_forward,1001189
flow_iat_min,2891
flow_packets/s,115
flow_duration,115
flow_iat_max,115
flow_iat_mean,115
flow_bytes/s,85
fwd_header_length,35
min_seg_size_forward,35


In [30]:
#Se decide que columnas eliminar con base en su porcentaje de muestras negativas

neg_conteo = (df[num_cols] < 0).sum().sort_values(ascending=False)

print(f"Cantidad de features previo a drop {len(df.columns)}")

print(f"El porcentaje de muestras de col negativa init_win_bytes_backward es {neg_conteo['init_win_bytes_backward'] / len(df)*100}")

print(f"El porcentaje de muestras de col negativa init_win_bytes_forward es {neg_conteo['init_win_bytes_forward'] / len(df)*100}")

#Tamaño inicial de la ventana TCP del host origen init_win_bytes_backward como bytes no puede ser negativo es ilogico asi que mejor elimino la feature
#Tamaño inicial de la ventana TCP del host destino init_win_bytes_forward como bytes no puede ser negativo es ilogico asi que mejor elimino la feature
#Ambas features tienen un porcentaje importante de valores negativos que sería complejo de manejar por ajuste o imputación además que acorde al análisis exploratorio (Ver DataExploration) no aportar a distinguir ataques por lo que se decide eliminarlas
#Además se reconoce que hay una columna duplicada fwd_header_length.1 por lo que la misma tambien se selecciona para su eliminacion

print(f"Las dos columnas fwd_header_length.1 y fwd_header_length son iguales en todas sus filas: {(df['fwd_header_length'] == df['fwd_header_length.1']).all()}")

cols_to_drop = ['init_win_bytes_backward','init_win_bytes_forward','fwd_header_length.1']

df = df.drop(columns=cols_to_drop)

print(f"Cantidad de features posterior a drop {len(df.columns)}")

Cantidad de features previo a drop 79
El porcentaje de muestras de col negativa init_win_bytes_backward es 50.92486318962901
El porcentaje de muestras de col negativa init_win_bytes_forward es 35.368417408433054
Las dos columnas fwd_header_length.1 y fwd_header_length son iguales en todas sus filas: True
Cantidad de features posterior a drop 76


In [ ]:
#Se analiza el resto de features con valores menores a cero para ver cuales tienen muestras solo a nivel de ataques y solo a nivel de trafico benigno

num_cols = df.select_dtypes(include="number").columns
neg_conteo = (df[num_cols] < 0).sum().sort_values(ascending=False)
cols_with_negatives = neg_conteo[neg_conteo > 0].index.tolist()

print(f"Todas las features con negativos {cols_with_negatives}")

ataques = df[df['label'] != 'BENIGN']

negatives_in_attacks = {}
negatives_in_benignos = []

for col in cols_with_negatives:
    count = (ataques[col] < 0).sum()
    if count > 0:
        negatives_in_attacks[col] = count
    else:
        negatives_in_benignos.append(col)

print(f"Features con negativos en muestras de ataques {negatives_in_attacks}") 

negatives_only_benignos=[]

for elem in negatives_in_benignos:
    if(elem not in negatives_in_attacks):
        negatives_only_benignos.append(elem)

print(f"Features con negativos solo en trafico benigno {negatives_only_benignos}")


Todas las features con negativos ['flow_iat_min', 'flow_iat_mean', 'flow_duration', 'flow_iat_max', 'flow_packets/s', 'flow_bytes/s', 'fwd_header_length', 'min_seg_size_forward', 'bwd_header_length', 'fwd_iat_min']
Features con negativos en muestras de ataques {'flow_iat_min': np.int64(194), 'fwd_iat_min': np.int64(17)}
Features con negativos solo en trafico benigno ['flow_iat_mean', 'flow_duration', 'flow_iat_max', 'flow_packets/s', 'flow_bytes/s', 'fwd_header_length', 'min_seg_size_forward', 'bwd_header_length']


In [32]:
#Remuevo muestras con negativos solo a nivel de clase mayoritaria

num_cols = df.select_dtypes(include="number").columns

antes = df["label"].value_counts()

mask_negativos = (df[negatives_in_benignos] < 0).any(axis=1)
print(f"Filas a ser removidas en clase mayoritaria de benignos: {mask_negativos.sum()}")

df_post_negs = df[~mask_negativos].copy()

despues = df_post_negs["label"].value_counts()

comparacion = pd.DataFrame({"antes": antes, "despues": despues}).fillna(0)
comparacion["muestras_perdidas"] = comparacion["antes"] - comparacion["despues"]

print(comparacion)

Filas a ser removidas en clase mayoritaria de benignos: 150
                             antes  despues  muestras_perdidas
label                                                         
BENIGN                     2273097  2272947                150
DoS Hulk                    231073   231073                  0
PortScan                    158930   158930                  0
DDoS                        128027   128027                  0
DoS GoldenEye                10293    10293                  0
FTP-Patator                   7938     7938                  0
SSH-Patator                   5897     5897                  0
DoS slowloris                 5796     5796                  0
DoS Slowhttptest              5499     5499                  0
Bot                           1966     1966                  0
Web Attack  Brute Force       1507     1507                  0
Web Attack  XSS                652      652                  0
Infiltration                    36       36               

In [33]:
from scipy.stats import chi2_contingency

#Aplico analisis estadistico con diferencia maxima de proporciones para verificar que remover estas muestras de la clase mayoritaria no ha afectado la distribucion

prop_antes = antes / antes.sum()
prop_despues = despues / despues.sum()

prop_despues = prop_despues.reindex(prop_antes.index).fillna(0)

diff_abs = (prop_antes - prop_despues).abs()

delta_max = diff_abs.max()

print("Diferencia máxima de proporciones:", delta_max)
print("\nCambios por clase:")
print(diff_abs.sort_values(ascending=False))

#delta_max < 0.001 indica que no hay difencia significativa en la distribucion (Ver F. Hinder et al., ACM Computing Surveys, 2023)

Diferencia máxima de proporciones: 1.0439314463317473e-05

Cambios por clase:
label
BENIGN                       1.043931e-05
DoS Hulk                     4.325762e-06
PortScan                     2.975221e-06
DDoS                         2.396707e-06
DoS GoldenEye                1.926883e-07
FTP-Patator                  1.486019e-07
SSH-Patator                  1.103938e-07
DoS slowloris                1.085030e-07
DoS Slowhttptest             1.029431e-07
Bot                          3.680416e-08
Web Attack  Brute Force      2.821153e-08
Web Attack  XSS              1.220565e-08
Infiltration                 6.739317e-10
Web Attack  Sql Injection    3.931268e-10
Heartbleed                   2.059236e-10
Name: count, dtype: float64


In [34]:
#Para el tratamiento de los valores con negativos en las columnas de ataques los convierto en NaN para posteriormente imputarlos mediante modelos ML

negatives_in_attack_list = list(negatives_in_attacks.keys())
df_post_negs[negatives_in_attack_list] = df_post_negs[negatives_in_attack_list].mask(df_post_negs[negatives_in_attack_list] < 0, np.nan)

In [35]:
#2. Manejo de valores nulos

#Se inicia contando la cantidad de negativos por feature en el dataset

print(df_post_negs.isnull().sum().sum())
cols_with_nans = df_post_negs.isna().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0]
print(cols_with_nans)

4151
flow_bytes/s    1358
flow_iat_min    2776
fwd_iat_min       17
dtype: int64


In [36]:
#Busco validar si de igual manera puedo eliminar las filas con nulls de clase benigna

cols_with_nans = df_post_negs.columns[df_post_negs.isna().any()]

antes = df_post_negs["label"].value_counts()

mask_nan_benign = (df_post_negs["label"] == "BENIGN") & df_post_negs[cols_with_nans].isna().any(axis=1)

print(f"Filas BENIGN con NaN a remover: {mask_nan_benign.sum()}")

df_post_negs_nan = df_post_negs[~mask_nan_benign].copy()

despues = df_post_negs_nan["label"].value_counts()

comparacion = pd.DataFrame({"antes": antes, "despues": despues}).fillna(0)
comparacion["muestras_perdidas"] = comparacion["antes"] - comparacion["despues"]

print(comparacion)

Filas BENIGN con NaN a remover: 2990
                             antes  despues  muestras_perdidas
label                                                         
BENIGN                     2272947  2269957               2990
DoS Hulk                    231073   231073                  0
PortScan                    158930   158930                  0
DDoS                        128027   128027                  0
DoS GoldenEye                10293    10293                  0
FTP-Patator                   7938     7938                  0
SSH-Patator                   5897     5897                  0
DoS slowloris                 5796     5796                  0
DoS Slowhttptest              5499     5499                  0
Bot                           1966     1966                  0
Web Attack  Brute Force       1507     1507                  0
Web Attack  XSS                652      652                  0
Infiltration                    36       36                  0
Web Attack  Sql In

In [ ]:
#Aplico analisis estadistico de igual manera tras esta eliminacion empleando chi cuadrado para verificar que remover estas muestras de la clase mayoritaria no ha afectado la distribucion

prop_antes = antes / antes.sum()
prop_despues = despues / despues.sum()

prop_despues = prop_despues.reindex(prop_antes.index).fillna(0)

diff_abs = (prop_antes - prop_despues).abs()

delta_max = diff_abs.max()

print("Diferencia máxima de proporciones:", delta_max)
print("\nCambios por clase:")
print(diff_abs.sort_values(ascending=False))

#delta_max < 0.01 indica que no hay difencia significativa en la distribucion (Ver F. Hinder et al., ACM Computing Surveys, 2023)

Diferencia máxima de proporciones: 0.00020832141537552307

Cambios por clase:
label
BENIGN                       2.083214e-04
DoS Hulk                     8.632260e-05
PortScan                     5.937194e-05
DDoS                         4.782741e-05
DoS GoldenEye                3.845186e-06
FTP-Patator                  2.965421e-06
SSH-Patator                  2.202959e-06
DoS slowloris                2.165228e-06
DoS Slowhttptest             2.054277e-06
Bot                          7.344443e-07
Web Attack  Brute Force      5.629743e-07
Web Attack  XSS              2.435695e-07
Infiltration                 1.344862e-08
Web Attack  Sql Injection    7.845030e-09
Heartbleed                   4.109302e-09
Name: count, dtype: float64


In [38]:
#Para el manejo de los nulos en el resto de muestras las cuales se muestran a continuacion se aplicara imputacion mediante modelos ML

cols_with_nans = df_post_negs_nan.isna().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0]
print(cols_with_nans)

flow_bytes/s    949
flow_iat_min    194
fwd_iat_min      17
dtype: int64


In [ ]:
#3. Analizo el manejo de duplicados

#La literatura en IDS describe específicamente la eliminación de duplicados como parte de la limpieza de datos para mejorar la calidad de entrenamiento del modelo ya que datos duplicados pueden introducir sesgo innecesario hacia la clase mayoritaria

df_benign = df_post_negs_nan[df_post_negs_nan["label"] == "BENIGN"]

df_malicious = df_post_negs_nan[df_post_negs_nan["label"] != "BENIGN"]

df_benign_unique = df_benign.drop_duplicates()

df_post_negs_nan_duplicates = pd.concat([df_benign_unique, df_malicious], ignore_index=True)

print(f"Registros eliminados en Benign: {len(df_benign) - len(df_benign_unique)}")

Registros eliminados en Benign: 222812


In [44]:
antes = df_post_negs_nan["label"].value_counts()
despues = df_post_negs_nan_duplicates["label"].value_counts()

comparacion = pd.DataFrame({"antes": antes,"despues": despues}).fillna(0).astype(int)

comparacion["muestras_perdidas"] = comparacion["antes"] - comparacion["despues"]

print(comparacion)

                             antes  despues  muestras_perdidas
label                                                         
BENIGN                     2269957  2047145             222812
DoS Hulk                    231073   231073                  0
PortScan                    158930   158930                  0
DDoS                        128027   128027                  0
DoS GoldenEye                10293    10293                  0
FTP-Patator                   7938     7938                  0
SSH-Patator                   5897     5897                  0
DoS slowloris                 5796     5796                  0
DoS Slowhttptest              5499     5499                  0
Bot                           1966     1966                  0
Web Attack  Brute Force       1507     1507                  0
Web Attack  XSS                652      652                  0
Infiltration                    36       36                  0
Web Attack  Sql Injection       21       21            

In [ ]:
#Aplico la prueba de chi cuadrado para igual validar si ha habido afectacion considerable en la distribucion

prop_antes = antes / antes.sum()
prop_despues = despues / despues.sum()

prop_despues = prop_despues.reindex(prop_antes.index).fillna(0)

diff_abs = (prop_antes - prop_despues).abs()
delta_max = diff_abs.max()

print("Diferencia máxima de proporciones:", delta_max)
print(diff_abs.sort_values(ascending=False))

#Con base en justificacion de que la eliminacion de datos duplicados es comun en tratamiento de sets de intrusiones se tolera cambio de 1.69% en proporcion

Diferencia máxima de proporciones: 0.016869641441049077
label
BENIGN                       1.686964e-02
DoS Hulk                     6.990310e-03
PortScan                     4.807875e-03
DDoS                         3.873012e-03
DoS GoldenEye                3.113789e-04
FTP-Patator                  2.401366e-04
SSH-Patator                  1.783932e-04
DoS slowloris                1.753378e-04
DoS Slowhttptest             1.663531e-04
Bot                          5.947450e-05
Web Attack  Brute Force      4.558905e-05
Web Attack  XSS              1.972399e-05
Infiltration                 1.089055e-06
Web Attack  Sql Injection    6.352820e-07
Heartbleed                   3.327668e-07
Name: count, dtype: float64


In [ ]:
#4. Manejo de valores iguales a 0

# En el dataset existen features que tienen valor de 0 a nivel de todas las muestras como las mismas no son informativas ni aportan al aprendizaje del modelo se eliminan

num_cols = df_post_negs_nan_duplicates.select_dtypes(include="number").columns

cero_conteo = (df_post_negs_nan_duplicates[num_cols] == 0).sum()

cero_conteo.name = "Conteo de Muestras por Feature"

features_todo_cero = cero_conteo[cero_conteo == len(df_post_negs_nan_duplicates)]

print(features_todo_cero)

bwd_psh_flags           2604791
bwd_urg_flags           2604791
fwd_avg_bytes/bulk      2604791
fwd_avg_packets/bulk    2604791
fwd_avg_bulk_rate       2604791
bwd_avg_bytes/bulk      2604791
bwd_avg_packets/bulk    2604791
bwd_avg_bulk_rate       2604791
Name: Conteo de Muestras por Feature, dtype: int64


In [56]:
df_post_negs_nan_duplicates_zero = df_post_negs_nan_duplicates.drop(columns=features_todo_cero.index)

In [57]:
print(f"Cantidad de features posterior a drop {len(df_post_negs_nan_duplicates_zero.columns)}")

Cantidad de features posterior a drop 68


In [63]:
#Nueva distribucion porcentual
df_post_negs_nan_duplicates_zero["label"].value_counts().to_frame("count").assign(
    percent=lambda x: 100 * x["count"] / x["count"].sum()
)

,count,percent
label,,
BENIGN,2047145,78.591526
DoS Hulk,231073,8.871076
PortScan,158930,6.101449
DDoS,128027,4.915058
DoS GoldenEye,10293,0.395156
FTP-Patator,7938,0.304746
SSH-Patator,5897,0.226391
DoS slowloris,5796,0.222513
DoS Slowhttptest,5499,0.211111


In [ ]:
#Nuevo ratio de desbalance
conteo = df_post_negs_nan_duplicates_zero["label"].value_counts()
conteo.max() / conteo.min()

186104.0909090909

In [ ]:
#Revision de nulos en dataset
null_conteo = df_post_negs_nan_duplicates_zero[num_cols].isnull().sum().sort_values(ascending=False)
print("Columnas con NaNs en dataset")
print(null_conteo[null_conteo > 0])

Columnas con NaNs en dataset
flow_bytes/s    949
flow_iat_min    194
fwd_iat_min      17
dtype: int64


In [71]:
#5. Para continuar con el preprocesamiento genero la particion de X, y
X=df_post_negs_nan_duplicates_zero.drop(columns=["label"])
y=df_post_negs_nan_duplicates_zero["label"]

In [72]:
#6. Manejo de valores infinitos
inf_per_column = np.isinf(X).sum()
infinite_cols=inf_per_column[inf_per_column > 0].sort_values(ascending=False)
print(infinite_cols)
print("Se verifica por conocimiento del dataset que dichas features tienen valores infinitos claramente derivados de calculos con divisores sumamente bajos")

flow_packets/s    1984
flow_bytes/s      1035
dtype: int64
Se verifica por conocimiento del dataset que dichas features tienen valores infinitos claramente derivados de calculos con divisores sumamente bajos


In [73]:
#Como es estandar en research para el tratamiento de los valores infinitos igual imputo el NaN y permito que el imputer aprenda para hacer el fill de esos valores
X = X.replace([np.inf, -np.inf], np.nan)

In [74]:
cols_with_nans = X.isna().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0]
print(f"Estas columnas tienen NaNs a imputar \n{cols_with_nans}")

Estas columnas tienen NaNs a imputar 
flow_bytes/s      1984
flow_packets/s    1984
flow_iat_min       194
fwd_iat_min         17
dtype: int64


In [76]:
#Exporto datasets X,y listos para entrenamientos por pipelines 
X.to_pickle("Sets_Xy/X.pkl")
y.to_pickle("Sets_Xy/y.pkl")